In [6]:
actions = (0,1,2,3)  # actions (0=up, 1=right,2=down,3=left)
states = (0, 1, 2, 3)  # states (tiles)
rewards = [-1, -1, -1, 10]  # Direct rewards per state
gamma = 0.9  
delta = 10**-2
# Transition probabilities per state-action pair
probs = [
    [[0.9,0.1,0,0], [0.1,0.8,0.1,0], [0.1,0.1,0.8,0], [0.9,0,0.1,0]],
    [[0.1,0.9,0,0], [0,0.9,0,0.1]  , [0.1,0.1,0,0.8], [0.8,0.1,0,0.1]],
    [[0.8,0,0.1,0.1], [0.1,0,0.1,0.8] , [0,0,0.9,0.1], [0.1,0,0.9,0]],
    [[0,0,0,0]    , [0,0,0,0]    , [0,0,0,0]    , [0,0,0,0]] # Term state (all probs 0)
]


max_policy_iter = 10000  # Maximum number of policy iterations
max_value_iter = 10000  # Maximum number of value iterations
pi = [0 for s in states]
V = [0 for s in states]

print(V,pi)
for i in range(max_policy_iter):

    optimal_policy_found = True

    # Policy evaluation
    # Compute value for each state under current policy
    for j in range(max_value_iter):
        max_diff = 0  # Initialize max difference
        V_new = [0, 0, 0, 0]  # Initialize values
        for s in states:

            # Compute state value
            val = rewards[s]  # Get direct reward
            for s_next in states:
                val += probs[s][s_next][pi[s]] * (
                        gamma * V[s_next]
                )  
            # if np.isnan(np.min(val)):
                # print("wtf")
      
            max_diff = max(max_diff, abs(val - V[s]))

            V[s] = val  # Update value with highest value
     
        if max_diff < delta:
            break


    for s in states:

        val_max = V[s] 
        for a in actions:
            val = rewards[s]  
            for s_next in states:
                val += probs[s][s_next][a] * (
                    gamma * V[s_next]
                )  

            # Update policy if (i) action improves value and (ii) action different from current policy
            if val > val_max and pi[s] != a:
                pi[s] = a
                val_max = val
                optimal_policy_found = False

    # If policy did not change, algorithm terminates
    if optimal_policy_found:
        break

    print(V,val_max,V_new,pi,a)

[0, 0, 0, 0] [0, 0, 0, 0]
[70.02109406689799, 17.168692837174138, 51.86037008351223, 10.0] 10.0 [0, 0, 0, 0] [0, 1, 2, 0] 3
wtf
wtf
wtf
wtf
wtf


In [2]:
import numpy as np
import random
import os
from copy import copy, deepcopy

#Extracts data from the Input txt File, assigns it to variables and returns the data to the main() function.
def read_inputfile():
    walls = []
    terminal_states = []
    terminal_state_reward = []
    transition_probabilities = []
    for files in os.listdir():
        filename, extension = os.path.splitext(files)
        if extension == '.txt':
            input_file = files
    with open(input_file) as f:
        lines = f.readlines()
    for line in lines:
        if ":" in line:
            split_line = line.split(":")
            if split_line[0].strip() == 'size':
                interim_size = split_line[1].strip()
                size = interim_size.split(" ")
                size = [int(size[0]), int(size[1])]
            if split_line[0].strip() == 'walls':
                interim_walls = split_line[1].strip()
                interim_walls = interim_walls.split(",")
                for interim_wall in interim_walls:
                    interim_wall = interim_wall.strip()
                    interim_wall = interim_wall.split(" ")
                    walls.append([int(interim_wall[0]), int(interim_wall[1])])
            if split_line[0].strip() == 'terminal_states':
                interim_terminalstates = split_line[1].strip()
                interim_terminalstates = interim_terminalstates.split(",")
                for interim_terminalstate in interim_terminalstates:
                    interim_terminalstate = interim_terminalstate.strip()
                    interim_terminalstate = interim_terminalstate.split(" ")
                    terminal_states.append([int(interim_terminalstate[0]), int(interim_terminalstate[1])])
                    terminal_state_reward.append(int(interim_terminalstate[2]))
            if split_line[0].strip() == 'reward':
                interim_reward = split_line[1].strip()
                reward = float(interim_reward)
            if split_line[0].strip() == 'transition_probabilities':
                interim_TP = split_line[1].strip()
                interim_TP = interim_TP.split(" ")
                for interim_tp in interim_TP:
                    interim_tp = interim_tp.strip()
                    transition_probabilities.append(float(interim_tp))
            if split_line[0].strip() == 'discount_rate':
                interim_discountrate = split_line[1].strip()
                discount_rate = float(interim_discountrate)
            if split_line[0].strip() == 'epsilon':
                interim_epsilon = split_line[1].strip()
                epsilon = float(interim_epsilon)
    return size, walls, terminal_states, terminal_state_reward, reward, transition_probabilities, discount_rate, epsilon

#Used to create the Initial State based on the mentioned Size of the Matrix, and the position of walls.
def create_initialstate(size, walls):
    num_rows, num_columns = size
    initial_state = np.zeros((num_columns, num_rows))
    for wall in walls:
        wall_row, wall_column = wall
        initial_state[wall_column-1][wall_row-1] = np.nan
    return initial_state

#Returns a Reward Matrix created based on the mentioned Matrix Size, Position of Terminal States, Correspnding Rewards and Transition Rewards
def create_rewardmatrix(size, transition_reward, terminal_states, terminal_state_reward):
    num_rows, num_columns = size
    reward = np.full((num_columns, num_rows), transition_reward)
    for value in range(len(terminal_states)):
        state_rows, state_columns = terminal_states[value]
        reward[abs(state_columns - num_columns)][abs(state_rows - 1)] = terminal_state_reward[value]
    return reward

#Returns the List of Possible Actions for Each Intended Action (taking into consideration the stochasticity)
def action_predictor(row, column, intended_action, transition_probabilities):
    intended_action = str(intended_action[0])
    TP = deepcopy(transition_probabilities)
    actions = ["U", "D", "L", "R"]
    if intended_action == "U":
        possible_actions = ["U", "L", "R"]
    if intended_action == "D":
        possible_actions = ["D", "R", "L"]
    if intended_action == "L":
        possible_actions = ["L", "U", "D"]
    if intended_action == "R":
        possible_actions = ["R", "U", "D"]
    corresponsing_probabilities = [TP[0], TP[1], TP[2]]
    return possible_actions, corresponsing_probabilities

#With the Position of Current Cell and Action as Inputs, the function returns the Next Cell index(taking into consideration Borders and Wall positions)
def action_outcome(initial_state, row, column, action):
    max_row, max_column = initial_state.shape
    max_row-=1
    max_column-=1
    if action == "U":
        if row == 0:
            next_row = row
            next_column = column
            return next_row, next_column
        elif np.isnan(initial_state[row - 1][column]):
            next_row = row
            next_column = column
            return next_row, next_column
        else:
            next_row = row - 1
            next_column = column
            return next_row, next_column
    if action == "D":
        if row == max_row:
            next_row = row
            next_column = column
            return next_row, next_column
        elif np.isnan(initial_state[row + 1][column]):
            next_row = row
            next_column = column
            return next_row, next_column
        else:
            next_row = row + 1
            next_column = column
            return next_row, next_column
    if action == "L":
        if column == 0:
            next_row = row
            next_column = column
            return next_row, next_column
        elif np.isnan(initial_state[row][column-1]):
            next_row = row
            next_column = column
            return next_row, next_column
        else:
            next_row = row
            next_column = column - 1
            return next_row, next_column
    if action == "R":
        if column == max_column:
            next_row = row
            next_column = column
            return next_row, next_column
        elif np.isnan(initial_state[row][column+1]):
            next_row = row
            next_column = column
            return next_row, next_column
        else:
            next_row = row
            next_column = column + 1
            return next_row, next_column

#Value Iteration function
def value_iteration(initial_state, reward_matrix, transition_probabilities, discount_rate, size, terminal_states, walls):
    print("-------------------------Value Iteration-------------------------")
    actions = ["U", "D", "L", "R"]
    current_state = deepcopy(initial_state)
    utility = deepcopy(initial_state)
    iterable_utility = deepcopy(initial_state)
    row_length, column_length = current_state.shape
    previous_utility = np.zeros((row_length, column_length))
    final_policy = np.zeros((row_length ,column_length), dtype='U1')
    num_rows, num_columns = size
    action_cell = []
    flag = True
    iteration = 0
    while flag == True:
        #Main Iteration Loop
        iteration = iteration + 1
        utility = deepcopy(iterable_utility)
        for row in range(row_length):
            for column in range(column_length):
                utility_cell = []
                action_tracker = []
                actionoutcome_tracker = []
                #cell_utility variable
                #Cell-wise Operation
                for intended_action in actions:
                    Q = 0
                    action_tracker.append(intended_action)
                    possible_actions, corresponsing_probabilities = action_predictor(row, column, intended_action, transition_probabilities)
                    for q_value in range(len(possible_actions)):
                        next_row, next_column = action_outcome(initial_state, row, column, possible_actions[q_value])
                        q_action = corresponsing_probabilities[q_value] * (reward_matrix[next_row][next_column] + (discount_rate * utility[next_row][next_column]))
                        Q = Q + q_action
                    utility_cell.append(Q)
                    actionoutcome_tracker.append(Q)
                iterable_utility[row][column] = max(utility_cell)
                max_action_index = actionoutcome_tracker.index(max(actionoutcome_tracker))
                final_policy[row][column] = action_tracker[max_action_index]
        for row in range(row_length):
            for column in range(column_length):
                if previous_utility[row][column] != iterable_utility[row][column]:
                    flag = True
                    continue
                else:
                    flag = False
        previous_utility = deepcopy(iterable_utility)
        for value in range(len(terminal_states)):
            state_rows, state_columns = terminal_states[value]
            iterable_utility[abs(state_columns - num_columns)][abs(state_rows - 1)] = 0
        print("\n")
        print("Iteration: ", iteration)
        print(iterable_utility)
    for value in range(len(terminal_states)):
        state_rows, state_columns = terminal_states[value]
        final_policy[abs(state_columns - num_columns)][abs(state_rows - 1)] = 'T'
    for wall in walls:
        wall_row, wall_column = wall
        final_policy[wall_column-1][wall_row-1] = '-'
    print("\n")
    print("Final Policy after Convergence: ")
    print(final_policy)
    return iterable_utility, final_policy

#Policy Iteration function
def policy_iteration(initial_state, reward_matrix, transition_probabilities, discount_rate, size, terminal_states, walls, epsilon):
    actions = ["U", "D", "L", "R"]
    current_state = deepcopy(initial_state)
    utility = deepcopy(initial_state)
    iterable_utility = deepcopy(initial_state)
    row_length, column_length = initial_state.shape
    previous_utility = np.zeros((row_length, column_length))
    num_rows, num_columns = size
    #Initializing Policy Matrix
    policy = np.zeros((row_length ,column_length), dtype='U1')
    random_action = np.random.randint(0, len(actions), (row_length, column_length))
    #Filling Random Actions in Policy to Create Initial Random Policy Matrix
    for row in range(row_length):
        for column in range(column_length):
            policy[row][column] = actions[random_action[row][column]]
    initial_random_policy = deepcopy(policy)
    outer_flag = True
    inner_flag = True
    optimal_utility, optimal_policy = value_iteration(initial_state, reward_matrix, transition_probabilities, discount_rate, size, terminal_states, walls)
    iteration = 0
    while outer_flag is True:
        iteration+=1
        epsilon_array = []
        #Iteration to compare Policy Values
        while inner_flag is True:
            #Value Iteration
            utility = deepcopy(iterable_utility)
            for row in range(row_length):
                for column in range(column_length):
                    intended_action = []
                    intended_action.append(policy[row][column])
                    utility_cell = []
                    for action in intended_action:
                        Q = 0
                        possible_actions, corresponsing_probabilities = action_predictor(row, column, intended_action, transition_probabilities)
                        for q_value in range(len(possible_actions)):
                            next_row, next_column = action_outcome(current_state, row, column, possible_actions[q_value])
                            q_action = corresponsing_probabilities[q_value] * (reward_matrix[next_row][next_column] + (discount_rate * utility[next_row][next_column]))
                            Q = Q + q_action
                        utility_cell.append(Q)
                    iterable_utility[row][column] = max(utility_cell)
            for value in range(len(terminal_states)):
                state_rows, state_columns = terminal_states[value]
                iterable_utility[abs(state_columns - num_columns)][abs(state_rows - 1)] = 0
            for row in range(row_length):
                for column in range(column_length):
                    if previous_utility[row][column] != iterable_utility[row][column]:
                        inner_flag = True
                        continue
                    else:
                        inner_flag = False
            previous_utility = deepcopy(iterable_utility)
            for value in range(len(terminal_states)):
                state_rows, state_columns = terminal_states[value]
                iterable_utility[abs(state_columns - num_columns)][abs(state_rows - 1)] = 0

        for row in range(row_length):
            for column in range(column_length):
                value_difference = optimal_utility[row][column] - iterable_utility[row][column]
                epsilon_array.append(value_difference)

        for row in range(row_length):
            for column in range(column_length):
                if optimal_utility[row][column] > iterable_utility[row][column]:
                    policy[row][column] = optimal_policy[row][column]
                else:
                    continue
        for row in range(row_length):
            for column in range(column_length):
                if policy[row][column] != optimal_policy[row][column]:
                    outer_flag = True
                    continue
                else:
                    outer_flag = False
    for value in range(len(terminal_states)):
        state_rows, state_columns = terminal_states[value]
        policy[abs(state_columns - num_columns)][abs(state_rows - 1)] = 'T'
    for wall in walls:
        wall_row, wall_column = wall
        policy[wall_column-1][wall_row-1] = '-'
    print("------------------------Policy Iteration-------------------------")
    print("\n Initial Random Policy: ")
    print(initial_random_policy)
    print("\n Final Policy: ")
    print(policy)

def main():
    #Obtains extracted data from the Input File
    size, walls, terminal_states, terminal_state_reward, reward, transition_probabilities, discount_rate, epsilon = read_inputfile()
    #Creating Initial State Array
    initial_state = create_initialstate(size, walls)
    #Creating Reward Matrix Array
    reward_matrix = create_rewardmatrix(size, reward, terminal_states, terminal_state_reward)
    #Performing Policy Iteration
    policy_iteration(initial_state, reward_matrix, transition_probabilities, discount_rate, size, terminal_states, walls, epsilon)

if __name__=="__main__":
    main()

-------------------------Value Iteration-------------------------


Iteration:  1
[[-0.04  -0.04  -0.04   1.592  0.   ]
 [-0.04  -0.04  -0.04   0.496  0.   ]
 [-0.04     nan  0.792  0.     0.496]
 [-0.04  -0.04  -0.04   0.792 -0.04 ]]


Iteration:  2
[[-0.074   -0.074    1.03576  1.76948  0.     ]
 [-0.074   -0.074    0.53732  0.74316  0.     ]
 [-0.074        nan  0.7852   0.       0.4926 ]
 [-0.074   -0.074    0.56248  0.7852   0.53732]]


Iteration:  3
[[-0.1029     0.6517368  1.2969582  1.8055744  0.       ]
 [-0.1029     0.3127976  0.7731576  0.9129186  0.       ]
 [-0.1029           nan  0.885483   0.         0.5416722]
 [-0.1029     0.3299064  0.6084888  0.885483   0.5814792]]


Iteration:  4
[[ 0.38568802  0.95272683  1.36375044  1.82307191  0.        ]
 [-0.127465    0.56773259  0.98524805  0.95750899  0.        ]
 [-0.127465           nan  0.90943994  0.          0.54542573]
 [ 0.16684335  0.42985647  0.68911604  0.89314728  0.65759631]]


Iteration:  5
[[0.6298032  1.0493138